In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f=open('/content/drive/My Drive/Colab Notebooks/WhatsApp Chat.txt')
lines=[]
category=[] #0-Amlan,1-Prasad,2-Mihir,3-Dheer
def cat(x):
  A=x
  if 'Amlan' in A:
      category.append(0)
  elif 'Prasad Ms18' in A:
      category.append(1)
  elif 'Mihir IISER' in A:
      category.append(2)
  else:
      category.append(3)
for line in f:
  line=line.strip()
  cat(line) 
  line=line.lower()
   
  lines.append(line.split(':')[-1])
lines[0:10]

[' whooo',
 ' lab ka pata karo jaldi',
 ' tu vineet se puchna',
 ' <media omitted>',
 ' my net kharaba as fuck',
 ' me woke up and had to go outside due to some work',
 ' iss baar lagta koi chuya nahi',
 ' <media omitted>',
 ' ohkay',
 ' 💋']

In [ ]:
import numpy as np
rep="1234567890[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]"
for i in range(len(lines)):
  for j in rep:
    if j in lines[i]:
      lines[i]=str(lines[i]).replace(j," ")
lines[0:10],category[0:10]      

([' whooo',
  ' lab ka pata karo jaldi',
  ' tu vineet se puchna',
  '  media omitted ',
  ' my net kharaba as fuck',
  ' me woke up and had to go outside due to some work',
  ' iss baar lagta koi chuya nahi',
  '  media omitted ',
  ' ohkay',
  ' 💋'],
 [1, 2, 0, 1, 1, 2, 0, 1, 2, 0])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
X= lines
y= category
count_vect = CountVectorizer(binary=True, max_features=1000)

X = count_vect.fit_transform(X).toarray()
y = np.array(category)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.5, random_state=0)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((20932, 1000), (20932,), (20933, 1000), (20933,))

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
predicted= clf.predict(X_test[0:1000,:])
accuracy_score(y_test[0:1000],predicted)

0.431

In [ ]:
n = X_train.shape[0] 
d = X_train.shape[1] 
K = 4 

psis = np.zeros([K,d])
phis = np.zeros([K])


for k in range(K):
    X_k = X_train[y_train == k]
    psis[k] = np.mean(X_k, axis=0)
    phis[k] = X_k.shape[0] / float(n)


print(phis)

[0.26310593 0.20539525 0.21568253 0.31581629]


In [ ]:

def nb_predictions(x, psis, phis):
    n, d = x.shape
    x = np.reshape(x, (1, n, d))
    psis = np.reshape(psis, (K, 1, d))
    

    psis = psis.clip(1e-14, 1-1e-14)
    
    logpy = np.log(phis).reshape([K,1])
    logpxy = x * np.log(psis) + (1-x) * np.log(1-psis)
    logpyx = logpxy.sum(axis=2) + logpy

    return logpyx.argmax(axis=0).flatten(), logpyx.reshape([K,n])

idx, logpyx = nb_predictions(X_train, psis, phis)
print(idx[:10])

[0 2 2 0 1 2 2 0 2 0]


In [ ]:
text=["😂 😂 😂 😂 😂 😂"]


X_new = count_vect.transform(text).toarray()
predicted, logpyx_new = nb_predictions(X_new, psis, phis)

names={0:'Amlan',1:'Prasad',2:"Mihir",3:'Dheer'}

print("{} is more likely to say the input text".format(names[predicted[0]]))
print("Input Text: ", text[0])
logpyx_new


Amlan is more likely to say the input text
Input Text:  😂 😂 😂 😂 😂 😂


array([[-4.31648439],
       [-5.14752575],
       [-6.0033571 ],
       [-5.18138534]])